### **Parse MeSH XML** 

Le MeSH 2019 comprend 29 351 descripteurs  
https://mesh.inserm.fr/FrenchMesh/presentation.htm

In [2]:
import xml.etree.ElementTree as ET
import re
import pandas as pd

In [3]:
tree = ET.parse('../fredesc2019.xml')
root = tree.getroot()

In [106]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [110]:
base_path = '../04-filtrage/MeSH/'
file_path = 'mesh_ids_fr-en.csv'

In [111]:
data = [{'mesh_id' : x.find('DescriptorUI').text.strip('\n'), \
         'label_fr' : x.find('DescriptorName').find('String').text.split('[')[0], \
         'label_en' : x.find('DescriptorName').find('String').text.split('[')[1].strip(']'), \
         'synonymes (en/fr)' : flatten([[term.find('String').text for term in concept.find('TermList').findall('Term')] for concept in x.find('ConceptList').findall('Concept')]) \
         } for x in root.findall('DescriptorRecord')]

pd.DataFrame(data).to_csv(file_path)
pd.DataFrame(data)

,mesh_id,label_fr,label_en,synonymes (en/fr)
0,D000001,A-23187,Calcimycin,"[Calcimycin, A-23187, A-23187, A 23187, A23187..."
1,D000002,Téméfos,Temefos,"[Temefos, Temephos, Téméfos, Abate, Difos]"
2,D000003,Abattoirs,Abattoirs,"[Abattoirs, Abattoir, Slaughter Houses, House,..."
3,D000004,Abréviations comme sujet,Abbreviations as Topic,"[Abbreviations as Topic, Abréviations comme su..."
4,D000005,Abdomen,Abdomen,"[Abdomen, Abdomens, Abdomen, Ventre]"
...,...,...,...,...
29346,D066310,Fracture numérique,Digital Divide,"[Digital Divide, Divide, Digital, Fracture num..."
29347,D066328,Striatum ventral,Ventral Striatum,"[Ventral Striatum, Striatum, Ventral, Striatum..."
29348,D066329,Agrégats de protéines,Protein Aggregates,"[Protein Aggregates, Aggregates, Protein, Agré..."
29349,D066330,Impression tridimensionnelle,"Printing, Three-Dimensional","[Printing, Three-Dimensional, Printing, Three ..."


In [109]:
for desc in root.findall('DescriptorRecord')[:5]:
    for concept in desc.find('ConceptList'):
        for term in concept.find('TermList').findall('Term'):
            print(term.find('String').text)

Calcimycin
A-23187
A-23187
A 23187
A23187
Antibiotic A23187
A23187, Antibiotic
Temefos
Temephos
Téméfos
Abate
Difos
Abattoirs
Abattoir
Slaughter Houses
House, Slaughter
Houses, Slaughter
Slaughter House
Slaughterhouses
Slaughterhouse
Abattoirs
Abattoir
Abbreviations as Topic
Abréviations comme sujet
Acronyms as Topic
Acronymes comme sujet
Abdomen
Abdomens
Abdomen
Ventre


In [80]:
strings = [x for x in root.findall('.//String')]

In [76]:
regex = '.+\[.*\]'
labels = pd.DataFrame([{'label_fr' : x.text.split('[')[0], 'label_en': x.text.split('[')[1].strip(']'), 'MeSHID' : None} for x in strings if re.match(regex, x.text)]).drop_duplicates()

Avec les synonymes, on a 47 176 termes 

In [77]:
labels

,label_fr,label_en,MeSHID
0,A-23187,Calcimycin,None
2,Téméfos,Temefos,None
4,Abattoirs,Abattoirs,None
6,Abréviations comme sujet,Abbreviations as Topic,None
8,Acronymes comme sujet,Acronyms as Topic,None
...,...,...,...
76539,Fracture numérique,Digital Divide,None
76541,Striatum ventral,Ventral Striatum,None
76543,Agrégats de protéines,Protein Aggregates,None
76545,Impression tridimensionnelle,"Printing, Three-Dimensional",None
